# OLS Baseline 
In this notebook an OLS estimation will be performed which serves as a baseline to compare the other methods with. It also serves as the proof-of-concept to figure out how to structure the data correctly and save results before moving on to other architectures.

In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
def createRollingWindow(dataset, look_back=1):
    X= pd.DataFrame(np.empty((dataset.shape[0]-look_back, dataset.shape[1]*look_back)))
    for i in tqdm(range(dataset.shape[0]-look_back)):    
        X.iloc[i] = dataset.iloc[i:(i+look_back):].to_numpy().flatten()
    return X

In [4]:
def shift_data(steps, X, y):
    X = X[:X.shape[0]-steps]
    y = y.shift(periods=-steps)[:y.shape[0]-steps].reset_index(drop=True)
    return X,y

## Reading Data
First we start with loading the relevant data from the excel to be used in our analyis

In [5]:
#Read the equity premium series to a dataframe
ep = pd.read_excel('data/Augemented_Formatted_results.xls', sheet_name='Equity premium', skiprows= range(1118,1127,1))[:-1]
ep['Date'] = pd.to_datetime(ep['Date'], format='%Y%m')
ep = ep.set_index('Date')
ep = ep.loc[(ep.index >= '1950-12-01')]

In [6]:
#Read the maacroeconomic variables to a dataframe
mev = pd.read_excel('data/Augemented_Formatted_results.xls', sheet_name='Macroeconomic variables', 
                    skiprows= range(1118,1126,1)).fillna(method='bfill')[:-1] #backward fill missing values. 
mev = mev.loc[:, ~mev.columns.str.match('Unnamed')]  #Remove empty column
mev['Date'] = pd.to_datetime(mev['Date'], format='%Y%m') #convert date pandas format
mev = mev.set_index('Date') #Set date as index. 
mev = mev.loc[(mev.index >= '1950-12-01')]

In [30]:
ta = pd.read_excel('data/Augemented_Formatted_results.xls', sheet_name='Technical indicators', 
                    skiprows= range(1118,1119,1))[:-1]
ta['Date'] = pd.to_datetime(ta['Date'], format='%Y%m')
ta = ta.set_index('Date')
ta = ta.loc[(ta.index >= '1950-12-01')]

### Data restructuring
We must create rolling windows of the Macro Economic Variables (MEV) and match them with the 1 month out of sample equity premium in order train a model. 

In [7]:
#Create rolling window version of the MEV dataset.  
X_mev = createRollingWindow(mev, look_back = 12)

In [8]:
#Shift equity premiumms such that they correspond to the 1 month out of sample corresponding to each window. 
y = ep.shift(periods=-12)[:ep.shape[0]-12].reset_index(drop=True)

#Convert y to a series with only log equity premium or simple equity premium 
y = y['Log equity premium'].astype('float64')

### Train OLS Model Windowed
Create rolling windows where we try to predit the 1 month out of sample equity premium based on the previous 12 months of Macro economic variables.

In [9]:
#Create Train and test set
X_train, X_test, y_train, y_test = train_test_split(X_mev, y, train_size=168, random_state=0, shuffle=False)

In [10]:
#Train a linear regression model on MEV rolling window data and the corresponding 1 month out of sample equity premium. 
reg = LinearRegression().fit(X_train, y_train)
coefficients = reg.coef_
intercept = reg.intercept_

In [11]:
#Make a prediction
y_pred = reg.predict(X_test)

from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2:', metrics.r2_score(y_test, y_pred))
print('Explained Variance:', metrics.explained_variance_score(y_test, y_pred))

Mean Absolute Error: 15.805841849587061
Mean Squared Error: 977.3759656738616
Root Mean Squared Error: 31.263012741478732
R2: -525094.7996636973
Explained Variance: -488032.71709039697


### Train OLS Model Vanilla
Train an OLS model without the rolling window variation. Here we just shift the equity premium by 1 such that we alling 1 row of MEV measurements with the 1 month out of sample equity premium. 

In [12]:
X = mev[:mev.shape[0]-1]
y = ep['Log equity premium'].shift(periods=-1)[:ep['Log equity premium'].shape[0]-1].reset_index(drop=True)

In [13]:
#Create Train and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=168, random_state=0, shuffle=False)

In [14]:
#Train a linear regression model on MEV rolling window data and the corresponding 1 month out of sample equity premium. 
reg = LinearRegression().fit(X_train, y_train)
coefficients = reg.coef_
intercept = reg.intercept_

In [15]:
#Make a prediction
y_pred = reg.predict(X_test)

from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('R2:', metrics.r2_score(y_test, y_pred))
print('Explained Variance:', metrics.explained_variance_score(y_test, y_pred))

Mean Absolute Error: 0.40171447999422094
Mean Squared Error: 0.2844314592484045
Root Mean Squared Error: 0.533321159573108
R2: -153.54061176469264
Explained Variance: -66.74475794944357


## WIP Notes
* What type of OLS regression should I run? Based on MEV and TA seperately I suppose? Perhaps read rapach 
* Right now I have contatenated all MEV into one big regression, I'm fairly sure I should do each regression seperately per variable.

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191
0,-2.630763,-2.585665,-1.972221,-0.658542,0.141845,0.722538,0.031083,1.34,2.24,0.16,0.90,0.53,0.07,0.406780,0.001117,2.840000,-2.678929,-2.619486,-2.032836,-0.646092,0.158199,0.683599,0.029776,1.34,2.21,0.58,0.87,0.51,-0.39,1.344110,0.001117,2.836670,-2.674227,-2.667785,-2.040457,-0.633770,0.156288,0.674866,0.030877,1.36,2.28,-0.74,0.92,0.50,0.30,1.589440,0.001133,2.833330,-2.644681,-2.663200,-2.023114,-0.621566,0.160400,0.781395,0.032442,1.40,2.41,-1.57,1.01,0.45,-0.80,1.175720,0.001167,2.830000,-2.682958,-2.635949,-2.083165,-0.599793,0.161481,0.749431,0.031849,1.47,2.48,-0.63,1.01,0.48,0.54,0.388850,0.001225,2.793330,-2.632879,-2.674295,-2.054959,-0.577919,0.156148,0.777889,0.030471,1.55,2.54,-0.69,0.99,0.51,0.54,0.129280,0.001292,2.756670,-2.597930,-2.624297,-2.041984,-0.555946,0.145999,0.800363,0.036027,1.45,2.59,-0.62,1.14,0.55,-0.31,0.386850,0.001208,2.720000,-2.672957,-2.606512,-2.134501,-0.538455,0.165429,0.753122,0.032369,1.56,2.52,1.38,0.96,0.59,0.67,-0.128780,0.001300,2.650000,-2.720153,-2.681619,-2.199805,-0.520348,0.166089,0.718594,0.030122,1.62,2.46,0.99,0.84,0.62,0.15,0.128780,0.001350,2.580000,-2.728025,-2.728884,-2.226452,-0.501572,0.145485,0.716182,0.028609,1.63,2.53,-0.80,0.90,0.62,0.23,0.000000,0.001358,2.510000,-2.738593,-2.752446,-2.221938,-0.516655,0.149349,0.740233,0.028911,1.54,2.54,0.10,1.00,0.61,-1.55,0.641440,0.001283,2.48667,-2.760999,-2.763618,-2.228749,-0.532250,0.145284,0.743292,0.031417,1.56,2.64,-1.36,1.08,0.60,0.75,0.510210,0.001300,2.46333
1,-2.678929,-2.619486,-2.032836,-0.646092,0.158199,0.683599,0.029776,1.34,2.21,0.58,0.87,0.51,-0.39,1.344110,0.001117,2.836670,-2.674227,-2.667785,-2.040457,-0.633770,0.156288,0.674866,0.030877,1.36,2.28,-0.74,0.92,0.50,0.30,1.589440,0.001133,2.833330,-2.644681,-2.663200,-2.023114,-0.621566,0.160400,0.781395,0.032442,1.40,2.41,-1.57,1.01,0.45,-0.80,1.175720,0.001167,2.830000,-2.682958,-2.635949,-2.083165,-0.599793,0.161481,0.749431,0.031849,1.47,2.48,-0.63,1.01,0.48,0.54,0.388850,0.001225,2.793330,-2.632879,-2.674295,-2.054959,-0.577919,0.156148,0.777889,0.030471,1.55,2.54,-0.69,0.99,0.51,0.54,0.129280,0.001292,2.756670,-2.597930,-2.624297,-2.041984,-0.555946,0.145999,0.800363,0.036027,1.45,2.59,-0.62,1.14,0.55,-0.31,0.386850,0.001208,2.720000,-2.672957,-2.606512,-2.134501,-0.538455,0.165429,0.753122,0.032369,1.56,2.52,1.38,0.96,0.59,0.67,-0.128780,0.001300,2.650000,-2.720153,-2.681619,-2.199805,-0.520348,0.166089,0.718594,0.030122,1.62,2.46,0.99,0.84,0.62,0.15,0.128780,0.001350,2.580000,-2.728025,-2.728884,-2.226452,-0.501572,0.145485,0.716182,0.028609,1.63,2.53,-0.80,0.90,0.62,0.23,0.000000,0.001358,2.510000,-2.738593,-2.752446,-2.221938,-0.516655,0.149349,0.740233,0.028911,1.54,2.54,0.10,1.00,0.61,-1.55,0.641440,0.001283,2.486670,-2.760999,-2.763618,-2.228749,-0.532250,0.145284,0.743292,0.031417,1.56,2.64,-1.36,1.08,0.60,0.75,0.510210,0.001300,2.46333,-2.824835,-2.786673,-2.276426,-0.548408,0.140300,0.721316,0.035921,1.73,2.69,-0.61,0.96,0.60,1.19,0.507620,0.001442,2.44000
2,-2.674227,-2.667785,-2.040457,-0.633770,0.156288,0.674866,0.030877,1.36,2.28,-0.74,0.92,0.50,0.30,1.589440,0.001133,2.833330,-2.644681,-2.663200,-2.023114,-0.621566,0.160400,0.781395,0.032442,1.40,2.41,-1.57,1.01,0.45,-0.80,1.175720,0.001167,2.830000,-2.682958,-2.635949,-2.083165,-0.599793,0.161481,0.749431,0.031849,1.47,2.48,-0.63,1.01

In [17]:
mev

,DP,DY,EP,DE,RVOL,BM,NTIS,TBL (ann %),LTY (ann %),LTR (%),TMS (ann %),DFY (ann %),DFR (%),INFL (%),Risk-free rate,12-month moving sum of earnings
Date,,,,,,,,,,,,,,,,
1950-12-01,-2.630763,-2.585665,-1.972221,-0.658542,0.141845,0.722538,0.031083,1.34,2.24,0.16,0.90,0.53,0.07,0.406780,0.001117,2.84000
1951-01-01,-2.678929,-2.619486,-2.032836,-0.646092,0.158199,0.683599,0.029776,1.34,2.21,0.58,0.87,0.51,-0.39,1.344110,0.001117,2.83667
1951-02-01,-2.674227,-2.667785,-2.040457,-0.633770,0.156288,0.674866,0.030877,1.36,2.28,-0.74,0.92,0.50,0.30,1.589440,0.001133,2.83333
1951-03-01,-2.644681,-2.663200,-2.023114,-0.621566,0.160400,0.781395,0.032442,1.40,2.41,-1.57,1.01,0.45,-0.80,1.175720,0.001167,2.83000
1951-04-01,-2.682958,-2.635949,-2.083165,-0.599793,0.161481,0.749431,0.031849,1.47,2.48,-0.63,1.01,0.48,0.54,0.388850,0.001225,2.79333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-08-01,-3.941330,-3.959588,-3.086025,-0.855305,0.185543,0.237917,-0.010244,1.95,1.63,7.97,-0.32,0.89,-0.59,0.167094,0.001625,133.69000
2019-09-01,-3.951689,-3.934654,-3.108987,-0.842702,0.190507,0.233377,-0.010959,1.89,1.70,-1.92,-0.19,0.88,0.02,-0.005067,0.001575,132.90000
2019-10-01,-3.965984,-3.945758,-3.112869,-0.853115,0.172367,0.232261,-0.013267,1.65,1.71,-0.52,0.06,0.91,0.58,0.078345,0.001375,135.09000
